### For this project, we have decide to use a pretrained model NLLB-200 and finetune it to fit our project needs.
Model link: https://huggingface.co/facebook/nllb-200-distilled-600M

In [16]:
#Import packages
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, TrainingArguments, Trainer, DataCollatorForSeq2Seq, pipeline
import numpy
import evaluate
import pandas
from sklearn.model_selection import train_test_split
from datasets import load_dataset, Dataset, DatasetDict

In [2]:
#Load model directly
tokenizer = AutoTokenizer.from_pretrained("facebook/nllb-200-distilled-600M", src_lang = 'en', tgt_lang = 'zh')
model = AutoModelForSeq2SeqLM.from_pretrained("facebook/nllb-200-distilled-600M")

We will be using 2 datasets from Amazon to finetune the NLLB-200 model. The source dataset will be in English and the target dataset will be in Chinese.

In [4]:
#load the source dataset
en_data = pandas.read_json('en-US.json', lines=True)
zh_data = pandas.read_json('zh-CN.json', lines=True)

data = pandas.concat([en_data['utt'], zh_data['utt']], axis=1, keys=['en', 'zh'])
train_data, test_data = train_test_split(data)
train_data, dev_data = train_test_split(train_data)

dataset = DatasetDict({
                        'train': Dataset.from_pandas(train_data),
                        'test': Dataset.from_pandas(test_data),
                        'dev': Dataset.from_pandas(dev_data)
                    })
dataset = dataset.remove_columns(["__index_level_0__"])
dataset


DatasetDict({
    train: Dataset({
        features: ['en', 'zh'],
        num_rows: 5061
    })
    test: Dataset({
        features: ['en', 'zh'],
        num_rows: 2250
    })
    dev: Dataset({
        features: ['en', 'zh'],
        num_rows: 1688
    })
})

We will be following the tutorials from Hugging Face to use PyTorch Trainer to finetune the pretrained model. https://huggingface.co/docs/transformers/en/training, https://huggingface.co/docs/transformers/tasks/translation

In [5]:
def tokenize_function(data):
    #return tokenizer(data['en'], data['zh'], truncation=True)
    padding = 'max_length'
    max_length = 100
    sources = [d for d in data['en']]
    targets = [d for d in data['zh']]
    inputs = tokenizer(sources, max_length=max_length, padding=padding, truncation=True)
    label = tokenizer(targets, max_length=max_length, padding=padding, truncation=True)
    inputs['labels'] = label['input_ids']
    return inputs

tokenized_dataset = dataset.map(tokenize_function, batched=True)
tokenized_dataset

Map: 100%|██████████| 1688/1688 [00:00<00:00, 18623.26 examples/s]


DatasetDict({
    train: Dataset({
        features: ['en', 'zh', 'input_ids', 'attention_mask', 'labels'],
        num_rows: 5061
    })
    test: Dataset({
        features: ['en', 'zh', 'input_ids', 'attention_mask', 'labels'],
        num_rows: 2250
    })
    dev: Dataset({
        features: ['en', 'zh', 'input_ids', 'attention_mask', 'labels'],
        num_rows: 1688
    })
})

In [6]:
metric = evaluate.load('accuracy')

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = numpy.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)

data_collator = DataCollatorForSeq2Seq(tokenizer=tokenizer,model=model)

In [7]:
training_args = TrainingArguments(
                    output_dir="Trained Model", 
                    evaluation_strategy="epoch"
                    )

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset['train'],
    eval_dataset=tokenized_dataset['test'],
    data_collator=data_collator,
    tokenizer=tokenizer
)


In [9]:
trainer.train()

 26%|██▋       | 500/1899 [58:26<2:34:04,  6.61s/it]Some non-default generation parameters are set in the model config. These should go into a GenerationConfig file (https://huggingface.co/docs/transformers/generation_strategies#save-a-custom-decoding-strategy-with-your-model) instead. This warning will be raised to an exception in v4.41.
Non-default generation parameters: {'max_length': 200}


{'loss': 2.4285, 'learning_rate': 3.683517640863612e-05, 'epoch': 0.79}


                                                      
 33%|███▎      | 633/1899 [1:21:44<2:13:30,  6.33s/it]

{'eval_loss': 0.11612840741872787, 'eval_runtime': 466.6431, 'eval_samples_per_second': 4.822, 'eval_steps_per_second': 0.604, 'epoch': 1.0}


 53%|█████▎    | 1000/1899 [2:03:11<1:40:42,  6.72s/it] Some non-default generation parameters are set in the model config. These should go into a GenerationConfig file (https://huggingface.co/docs/transformers/generation_strategies#save-a-custom-decoding-strategy-with-your-model) instead. This warning will be raised to an exception in v4.41.
Non-default generation parameters: {'max_length': 200}


...

 79%|███████▉  | 1500/1899 [3:07:14<44:32,  6.70s/it]    Some non-default generation parameters are set in the model config. These should go into a GenerationConfig file (https://huggingface.co/docs/transformers/generation_strategies#save-a-custom-decoding-strategy-with-your-model) instead. This warning will be raised to an exception in v4.41.
Non-default generation parameters: {'max_length': 200}


{'loss': 0.0856, 'learning_rate': 1.0505529225908373e-05, 'epoch': 2.37}


                                                     
100%|██████████| 1899/1899 [3:59:35<00:00,  7.57s/it]

{'eval_loss': 0.10698454082012177, 'eval_runtime': 470.4405, 'eval_samples_per_second': 4.783, 'eval_steps_per_second': 0.599, 'epoch': 3.0}
{'train_runtime': 14375.3035, 'train_samples_per_second': 1.056, 'train_steps_per_second': 0.132, 'train_loss': 0.7063732468623622, 'epoch': 3.0}


TrainOutput(global_step=1899, training_loss=0.7063732468623622, metrics={'train_runtime': 14375.3035, 'train_samples_per_second': 1.056, 'train_steps_per_second': 0.132, 'train_loss': 0.7063732468623622, 'epoch': 3.0})

Once we finished training our model, it is saved to the Trained Model folder. However, due to the size limit of GitHub, the final version of the FineTuned Model is pushed to a Hugging Face repo so that we can easily reference it for further usage.

In [10]:
trainer.save_model('./Trained Model/FineTuned Model')

Some non-default generation parameters are set in the model config. These should go into a GenerationConfig file (https://huggingface.co/docs/transformers/generation_strategies#save-a-custom-decoding-strategy-with-your-model) instead. This warning will be raised to an exception in v4.41.
Non-default generation parameters: {'max_length': 200}


The next step is to test our fine tune model to make sure everything is working correctly.

In [84]:
translator = pipeline("translation", model="duongy18418/Multilingual_Model")

In [114]:
ecommerce_dataset = pandas.read_csv("Amazon_Ecommerce_Data_2020.csv", usecols=["Category"], nrows=100)
ecommerce_dataset = ecommerce_dataset.dropna()
en_list = ecommerce_dataset['Category'].tolist()
zh_list = []

In [116]:
for i in range(len(en_list)):
    zh_list.append(translator(en_list[i], src_lang='en', tgt_lang='zh'))

In [123]:
ecommerce_dataset = pandas.DataFrame(list(zip(en_list, zh_list)), columns=['Category-en', 'Category-zh'])
print(ecommerce_dataset.to_markdown())

|    | Category-en                                                                                                                                         | Category-zh                                                                         |
|---:|:----------------------------------------------------------------------------------------------------------------------------------------------------|:------------------------------------------------------------------------------------|
|  0 | Sports & Outdoors | Outdoor Recreation | Skates, Skateboards & Scooters | Skateboarding | Standard Skateboards & Longboards | Longboards            | [{'translation_text': 'zheng 运动和户外娱乐滑板滑板滑板滑板滑板长板'}]              |
|  1 | Toys & Games | Learning & Education | Science Kits & Toys                                                                                           | [{'translation_text': 'zheng 玩具游戏学习教育科学套件玩具'}]                        |
|  2 | Toys & Games | Arts & Crafts | Craft Kits             